In [2]:
import math
import requests
import json

################################## GET ENVIROMENT TYPE ###################################

# Define the coordinate to check
lat = 52.006251
lon = 4.356806

# Build the OSM API query URL
url = 'https://nominatim.openstreetmap.org/reverse?format=json&lat={}&lon={}&zoom=18&addressdetails=1'.format(lat, lon)

# Send a GET request to the API and get the response
response = requests.get(url)

# Parse the response JSON data
data = json.loads(response.content)

# Check type
if 'city' in data['address']:
    print('The coordinates are in the city of', data['address']['city'])
    enviroment = "city"
elif 'village' in data['address']:
    print('The coordinates are in the village of', data['address']['village'])
    enviroment = "village"
else:
    print('The coordinates are in the plains.')
    enviroment = "plains"

################################### GET COLUMN PROPERTIES ########################################
# Loaded info
material = "concrete"
Length = 3 # m
Height = 0.8 # m
Width = 0.8 # m
Thickness = 0.032 # m

########################## GET WATER TYPE AND FLOOD DEPTH + FLOOD RISK #########################
WaterType = "salt" # majority of seas and rivers in the netherlands are salty
DesignStillwaterDepth = 1 # m

The coordinates are in the city of Delft


In [3]:
################################### GET VALUES FROM GIVEN VARIABLES ################################
#  Modulus of elasticity in MPa
match material:
    # Source: https://www.calculand.com/unit-converter/stoffe-liste.php?gruppe=Elastic+modulus+%28E%29%2C+Young%27s+modulus&einheit=1e6--MPa
    case "concrete":
        Elasticity =  24000 * 1000 # MPA concrete to kn/m2
    case "wood":
        Elasticity =  10000 * 1000 # MPA wood to kn/m2
    case "steel":
        Elasticity =  210000 * 1000 # MPA steel to kn/m2

# Second moment of area in m4
match material:
    case "concrete":
        Inertia = (((Width * (Height**3))) / 12 ) #
    case "wood":
        Inertia = ((Width * (Height**3)) / 12) #  
    case "steel":
        # Inertia = 14920 * 10**-8 # cm^4 to m4 steel HE 320A
        Inertia = ((Thickness * (Height**3)) / 12) + ((Width / 12)* ((Height**3)) - ((Height**3)*(1/5))) * -1

# specific weights per water type
match WaterType:
    case "salt":
        SpecificWeightWater = 64.0 * 16.0185 # lb/ft3 to kg/m3
    case "fresh":
        SpecificWeightWater = 62.4 * 16.0185 # lb/ft3 to kg/m3

# Debris weight
match enviroment:
    case "city":
        DebrisWeight =  1500 # car weight
    case "village":
        DebrisWeight =  655 # tree weight https://www.researchgate.net/figure/Summary-of-tree-heights-weights-and-volumes_tbl1_232354725
    case "plains":
        DebrisWeight =  454 # wood weight

# t = duration of impact in seconds
match material:
# City of Honolulu building code for impact durations
    case "concrete":
        impacttime = 0.1 # second
    case "wood":
        impacttime = 1 # second 
    case "steel":
        impacttime = 0.5 # second

# Max deflection limits Column
match material:
     # Source: Eurocode 5, 3, 2 (in order)
    case "concrete":
        PermittedDeflection =  (Length * 1000) / 500 # EUC 5 Concrete
    case "wood":
        PermittedDeflection =  (Length * 1000) / 300 # EUC 3 wood 
    case "steel":
        PermittedDeflection =  (Length * 1000) / 500 # EUC 2 steel

match material:
     # Source: Eurocode 
    case "concrete":
        AllowableShearStress = 0.5 * 1000 # EUC 2, MPa to kN/m2
    case "wood":
        AllowableShearStress = 0.5 * 1000 # EUC 5, MPa to kN/m2
    case "steel":
        AllowableShearStress =  140 * 1000 # EUC 3, MPa to kN/m2     

# CrossSectionalArea
match material:
     # Source: Eurocode 5, 3, 2 (in order)
    case "concrete":
        CrossSectionalArea = Height * Width # m2
    case "wood":
        CrossSectionalArea = Height * Width # m2
    case "steel":
        CrossSectionalArea =  86.82 / 10000 # cm2 to m2 [PLACEHOLDER] steel HE 260A

match material:
     # Source: Eurocode 
    case "concrete":
        AllowableBendingStress = 5 * 1000 # EUC 2, MPa to kN/m2
    case "wood":
        AllowableBendingStress = 10 * 1000 # EUC 5, MPa to kN/m2
    case "steel":
        AllowableBendingStress =  235 * 1000 # EUC 3, MPa to kN/m2  

In [5]:
GravitationalConstant = 9.81 # m/s
Velocity = (GravitationalConstant * DesignStillwaterDepth)**0.5 # V = (gds)0.5

# Breaking wave calculation
def breakingwave():
    DragCoefficient = 2.25 # if square
    HeightMetric = Height * 0.3048
    BreakingWaveOnPiles = 0.5 * DragCoefficient * SpecificWeightWater * HeightMetric * (DesignStillwaterDepth**2)
    return BreakingWaveOnPiles

# Hydrostatic
def hydrostatic():
    HydrostaticLoadPerWidth = 0.5 * SpecificWeightWater * (DesignStillwaterDepth**2) # kg
    HydrostaticLoad = HydrostaticLoadPerWidth * Width
    return HydrostaticLoad

# Couldn't convert to metric, thus this function will be in imperial
# Hydrodynamic (Fluid dynamics)
def hydrodynamic():
    CrossArea = Height * Width
    DragCoefficient = 2 # square
    match WaterType:
        case "salt":
            MassDensityFluid = 1.99  # slugs/ft3 
        case "fresh":
            MassDensityFluid = 1.94  # slugs/ft3 
    VelocityImp = Velocity * 3.2808399
    CrossAreaImp = CrossArea * 10.7639
    HydrodynamicLoad = 0.5 * DragCoefficient * MassDensityFluid * (VelocityImp**2) * CrossAreaImp
    # HydrodynamicLoad = HydrodynamicLoad * 0.453592
    return HydrodynamicLoad

def debrisimpact():
    DebrisLoad = (DebrisWeight * Velocity) / (GravitationalConstant * impacttime)
    return DebrisLoad

# Calculate all necessary loads
BreakingWaves = breakingwave()
HydrodynamicLoad = hydrodynamic()
HydrostaticLoad = hydrostatic()
DebrisLoad = debrisimpact()

print("breaking", BreakingWaves, "hydrodynamic", HydrodynamicLoad, "hydrostatic", HydrostaticLoad, "debris", DebrisLoad)

# convert weight into forces
LoadBreak = (BreakingWaves * GravitationalConstant) / 1000 # Fz (kN) = m*g / 1000 
LoadCombination = ((HydrodynamicLoad + HydrostaticLoad) * GravitationalConstant) / 1000 # Fz (kN) = m*g / 1000 
LoadDebris = (DebrisLoad * GravitationalConstant) / 1000 # Fz (kN) = m*g / 1000 

print("breaking", LoadBreak, "hydrodynamic", LoadCombination, "Debris", LoadDebris)

# Calculate reaction loads
def reactionloads(Load):
    ReactionloadR1 = ((Load * (DesignStillwaterDepth * 1/3)**2) / (2* Length**3)) * ((DesignStillwaterDepth*2/3) + (2*Length))
    ReactionloadR2 = ((Load * (DesignStillwaterDepth * 2/3)) / (2* (Length**3))) * ((3 * (Length**2)) - ((DesignStillwaterDepth*2/3)**2))
    return ReactionloadR1, ReactionloadR2

ReactionLoadsBreak = reactionloads(LoadBreak)
print("Breaking Wave:", "R1", ReactionLoadsBreak[0], "kN", "R2", ReactionLoadsBreak[1], "kN")

ReactionLoadsComb = reactionloads(LoadCombination)
print("Hydrostatic and Hydrodynamic:", "R1", ReactionLoadsComb[0], "kN", "R2", ReactionLoadsComb[1], "kN")

ReactionLoadsDebris = reactionloads(LoadDebris)
print("Debris Load:", "R1", ReactionLoadsComb[0], "kN", "R2", ReactionLoadsComb[1], "kN")

# Calculate moments for breaking loads and load combinations
def moments(Load, ReactionLoads):
    # formulas from structx
    MomentLoad = ReactionLoads[0] * (DesignStillwaterDepth * 2/3)
    MomentFixed = ((Load * (DesignStillwaterDepth * 2/3) * (DesignStillwaterDepth * 1/3)) / (2 * (Length**2))) * ((DesignStillwaterDepth * 2/3) + Length)
    return MomentLoad, MomentFixed

MomentLoadsBreak = moments(LoadBreak, ReactionLoadsBreak)
print("Breaking Wave:", "M1", round(MomentLoadsBreak[0], 3), "kN", "M2", round(MomentLoadsBreak[1], 3), "kN")

MomentLoadsCombination = moments(LoadCombination, ReactionLoadsComb)
print("Hydrostatic and Hydrodynamic:", "M1", MomentLoadsBreak[0], "kN", "M2", MomentLoadsBreak[1], "kN")

MomentLoadsDebris = moments(LoadDebris, ReactionLoadsDebris)
print("Debris impact:", "M1", MomentLoadsBreak[0], "kN", "M2", MomentLoadsBreak[1], "kN")

# calculate max deflection for both breaking loads and the load combination
def deflection(Load):
    # formulas from structx
    if (DesignStillwaterDepth * 2/3) < (0.414 * Length):
        MaxDeflection = (Load * (DesignStillwaterDepth * 2/3) / (3 * Elasticity * Inertia)) * (((Length**2) - ((DesignStillwaterDepth * 2/3)**2))**3) / ((3 * (Length**2) - ((DesignStillwaterDepth * 2/3)**2))**2)
    elif (DesignStillwaterDepth * 2/3) > (0.414 * Length):
        MaxDeflection = ((Load * (DesignStillwaterDepth * 2/3) * ((DesignStillwaterDepth * 1/3)**2)) / (6 * Elasticity * Inertia)) * math.sqrt((DesignStillwaterDepth * 2/3) / ((2*Length)+(DesignStillwaterDepth * 2/3)))
    return MaxDeflection

def deflection(Load):
    # formulas from structx
    if (DesignStillwaterDepth * 2/3) < (0.414 * Length):
        MaxDeflection = (Load * (DesignStillwaterDepth * 2/3) / (3 * Elasticity * Inertia)) * (((Length**2) - ((DesignStillwaterDepth * 2/3)**2))**3) / ((3 * (Length**2) - ((DesignStillwaterDepth * 2/3)**2))**2)
    elif (DesignStillwaterDepth * 2/3) > (0.414 * Length):
        MaxDeflection = ((Load * (DesignStillwaterDepth * 2/3) * ((DesignStillwaterDepth * 1/3)**2)) / (6 * Elasticity * Inertia)) * (math.sqrt((DesignStillwaterDepth * 2/3) / ((2*Length)+(DesignStillwaterDepth * 2/3))))
    return MaxDeflection

MaxdefBreak = deflection(LoadBreak) 
MaxdefCombination = deflection(LoadCombination) 
MaxdefDebris = deflection(LoadDebris) 

print(MaxdefBreak)
print(MaxdefCombination)

# check if max deflection exceeds permitted deflection
def delfectioncheck(MaxDeflection):
    if MaxDeflection < PermittedDeflection:
        print("Deflection: Risk detected", "permitted amount is", round(PermittedDeflection, 3), "mm occuring amount is", round(MaxDeflection * 1000, 3), "mm")
        probability = False
    else:
        print("Deflection: Column is OK", "permitted amount is", round(PermittedDeflection, 3), "mm occuring amount is", round(MaxDeflection * 1000, 3), "mm")
        probability = True
    return probability 

probabilitysum = 0

print("Deflection check for Breaking Loads...")
MaxdefBreak = MaxdefBreak * 1000
if delfectioncheck(MaxdefBreak) == False:
    probabilitysum = +1

print("Deflection check for Hydrostatic and Hydrodynamic Loads...")
MaxdefCombination = MaxdefCombination * 1000
if delfectioncheck(MaxdefCombination) == False:
    probabilitysum = +1

print("Deflection check for debris Loads...")
MaxdefDebris = MaxdefDebris * 1000
if delfectioncheck(MaxdefDebris) == False:
    probabilitysum = +1

print(probabilitysum)

# # Shear Stress τ = V / A
# def shearforce(Load):
#     ShearStress = Load / CrossSectionalArea
#     return ShearStress

# ShearStress = shearforce()
# # check if shear stress exceeds alloweable shear stress
# if ShearStress > AllowableShearStress:
#     print("Shear Stress: Risk detected", "permitted amount is", round(AllowableShearStress, 3), "kN/m2 occuring amount is", round(ShearStress, 3), "kN/m2")
# else:
#     print("Shear Stress: Column is OK", "permitted amount is", round(AllowableShearStress, 3), "kN/m2 occuring amount is", round(ShearStress, 3), "kN/m2")   

# # Bending Stress σ = My/I
# if MomentLoads[0] > MomentLoads[1]:
#     MomentLoads = MomentLoads[0]
# else: 
#     MomentLoads = MomentLoads[1]
# BendingStress = (MomentLoads * (Height/2)) / Inertia

# # check if bending stress exceeds alloweable bending stress
# if BendingStress > AllowableBendingStress:
#     print("Bending Stress: Risk detected", "permitted amount is", round(AllowableBendingStress, 3), "kN/m2 occuring amount is", round(BendingStress, 3), "kN/m2")
# else:
#     print("Bending Stress: Column is OK", "permitted amount is", round(AllowableBendingStress, 3), "kN/m2 occuring amount is", round(BendingStress, 3), "kN/m2")  

breaking 281.22847488 hydrodynamic 1447.5773799353865 hydrostatic 410.0736 debris 4789.131426105756
breaking 2.7588513385728 hydrodynamic 18.22355611316614 Debris 46.981379290097465
Breaking Wave: R1 0.037844325632 kN R2 0.9044793826048001 kN
Hydrostatic and Hydrodynamic: R1 0.24998019359624332 kN R2 5.974526626950216 kN
Debris Load: R1 0.24998019359624332 kN R2 5.974526626950216 kN
Breaking Wave: M1 0.025 kN M2 0.125 kN
Hydrostatic and Hydrodynamic: M1 0.025229550421333333 kN M2 0.1248862745856 kN
Debris impact: M1 0.025229550421333333 kN M2 0.1248862745856 kN
6.64599034183366e-07
4.390000150737371e-06
Deflection check for Breaking Loads...
Deflection: Risk detected permitted amount is 6.0 mm occuring amount is 0.665 mm
Deflection check for Hydrostatic and Hydrodynamic Loads...
Deflection: Risk detected permitted amount is 6.0 mm occuring amount is 4.39 mm
Deflection check for debris Loads...
Deflection: Risk detected permitted amount is 6.0 mm occuring amount is 11.318 mm
1


In [42]:
Inertia = 14920 * 10**-8 # cm^4 to m4 steel HE 320A
print(Inertia)
Inertia = ((Thickness * (Height**3)) / 12) + ((Width / 12)* ((Height**3)) - ((Height**3)*(1/5))) * -1
print(Inertia)

0.0001492
0.06963200000000003
